## Importing Libraries

In [30]:
import pandas as pd 
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import ast
import faiss
import pickle
from sentence_transformers import SentenceTransformer


In [31]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [32]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nadyinky/sephora-products-and-skincare-reviews")

print("Path to dataset files:", path)

100%|██████████| 147M/147M [00:34<00:00, 4.43MB/s] 

Extracting files...


Path to dataset files: /Users/zainabshakruwala/.cache/kagglehub/datasets/nadyinky/sephora-products-and-skincare-reviews/versions/2


In [45]:
# products = pd.read_csv(path + "/product_info.csv")
file_name = "/reviews_0-250.csv"
reviews = pd.read_csv(path + file_name)
print(reviews.columns)
reviews.drop(columns=reviews.columns[0], axis=1, inplace=True)
print(reviews.columns)
reviews.to_csv(path + file_name,index=False)

/var/folders/lr/r2t5zqq959sd0hhfx1kbj5_r0000gn/T/ipykernel_34576/843273941.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv(path + file_name)


Index(['Unnamed: 0', 'author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')
Index(['author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')


In [44]:
reviews.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.00,1.00,2,0,2,2023-02-01,"I use this with the Nudestix “Citrus Clean Balm & Make-Up Melt“ to double cleanse and it has completely changed my skin (for the better). The make-up melt is oil based and removes all of your makeup super easily. I follow-up with this water based cleanser, and I also use this just by itself when I’m not wearing make-up. It leaves the skin gently cleansed, but without stripping the skin. 10/10 recommend combining with the make-up melt. It’s perfection!",Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.00
1,1,31423088263,1,0.00,NaN,0,0,0,2023-03-21,"I bought this lip mask after reading the reviews and the hype. Unfortunately, it did not meet my expectations as vaseline petroleum jelly works way better for me.",Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitamin C,LANEIGE,24.00
2,2,5061282401,5,1.00,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited to get into bed and apply this lip mask. I do see a difference because I suffer from dry cracked lips. I drink a lot of water and apply lip balm daily but nothing helped until I started using this. untiluntistafted usinf this.,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitamin C,LANEIGE,24.00
3,3,6083038851,5,1.00,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time. I honestly don’t even use it for night time. I use it as an everyday lip balm. I love the texture. Gummy Bear is my second most favourite scent. Grapefruit is the best in my opinion.,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitamin C,LANEIGE,24.00
4,4,47056667835,5,1.00,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must have. After a few weeks of use I have learned I will always have by my bedside. I thought it was a little expensive but a little goes a long way. It is worth the price.",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitamin C,LANEIGE,24.00


## Preprocessing Data

In [155]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

In [156]:
products['rating'] = products['rating'].round(1)
products['rating'].head()

0   3.60
1   4.20
2   4.20
3   4.50
4   3.20
Name: rating, dtype: float64

In [157]:
products['on_sale'] = products['sale_price_usd'].apply(lambda x: 0 if pd.isna(x) else 1)

In [158]:
def clean_list(field_value):
    try:
        if isinstance(field_value, str) and field_value.startswith('['):
            items = ast.literal_eval(field_value)
            if isinstance(items, list):
                return ', '.join(str(i) for i in items)
            return str(field_value)
    except:
        return str(field_value)[:200]

In [159]:
def prepare_document(products):
    documents= []
    for idx, row in products.iterrows():
        variation_info = (
            f"{row['variation_value']} - {row['variation_desc']}"
            if pd.notna(row.get('variation_type'))
            else "standard size only"
        )
        ingredients = clean_list(row.get('ingredients', 'Not specified'))
        description = clean_list(row.get('highlights', ''))
        doc_text = f"""
Product Name: {row['product_name']}
Brand Name: {row["brand_name"]}
Ingredients: {ingredients}
On Sale: {'Yes' if row['on_sale'] == 1 else 'No'}
Reviews: {row.get('reviews', 0)} reviews
New: {'Yes' if row['new'] == 1 else 'No'}
Limited Edition: {'Yes' if row['limited_edition'] == 1 else 'No'}
Product Description: {description}
Variations Available: {variation_info}""".strip()
        metadata = {
            "product_id": row['product_id'],
            "brand_id": row['brand_id'],
            "rating" : row['rating'],
            "category" : row['primary_category'],
            "product_name": row['product_name'],
            "brand_name": row['brand_name'],
            "price_usd": row['sale_price_usd'] if row['on_sale'] == 1 else row['price_usd']
           
        }
        documents.append({
            "text": doc_text,
            "metadata": metadata
        })
    return documents

In [160]:
documents = prepare_document(products)

In [161]:
documents[0]

{'text': 'Product Name: Fragrance Discovery Set\nBrand Name: 19-69\nIngredients: Capri Eau de Parfum:, Alcohol Denat. (SD Alcohol 39C), Parfum (Fragrance) D-Limonene, Linalool, Benzyl Salicylate, Ethylhexyl Methoxycinnamate, Butyl Methoxydibenzoylmethane, Ethylhexyl Salicylate, Benzl Benzoate, Citral, Geraniol, Eugenol, Benzyl Alcohol, Farnesol, Citronellol, Isoeugenol., Invisible Post Eau de Parfum:, Alcohol Denat. (SD Alcohol 39C), Parfum (Fragrance) Ethylhexyl Methoxycinnamate, Ethylhexyl Salicylate, Butyl Methoxydibenzoylmethane, Benzyl Benzoate, Citral, Coumarin, Limonene, Alpha-Isomethyl Ionone, Linalool., Kashbah Eau de Parfum:, Alcohol Denat. (SD Alcohol 39C), Parfum (Fragrance) Coumarin, Ethylhexyl Methoxycinnamate, Butyl Methoxydibenzoylmethane, Ethylhexyl Salicylate, D-Limonene, Eugenol, Linalool, Citronellol, Geraniol, Cinnamal, Citral., L’Air Barbes Eau de Parfum:, Alcohol Denat. (SD Alcohol 39C), Parfum (Fragrance) Benzyl Salicylate, D-Limonene, Ethylhexyl Methoxycinnamat

In [162]:
texts = [doc['text'] for doc in documents]
metadata_list = [doc['metadata'] for doc in documents]

## Making Embeddings

In [163]:


embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


In [164]:
print("Creating Embeddings...")

embeddings = embedding_model.encode(
    texts,
    show_progress_bar=True,
    batch_size=23
)
print(f"✅ Created {len(embeddings)} embeddings")
print(f"Embedding dimension: {embeddings.shape[1]}")

Creating Embeddings...


Batches: 100%|██████████| 370/370 [02:08<00:00,  2.88it/s]

✅ Created 8494 embeddings
Embedding dimension: 384


## Vector Store

In [165]:

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings.astype('float32'))
print(f"FAISS index created with {index.ntotal} vectors")

faiss.write_index(index, "sephora_faiss.index")

# Save documents and metadata for retrieval
with open('sephora_documents.pkl', 'wb') as f:
    pickle.dump({
        'documents': documents,  # Full structure with text + metadata
        'texts': texts,          # Just texts (useful for display)
        'embedding_model_name': 'all-MiniLM-L6-v2'  # Track which model you used
    }, f)

print("✅ Saved FAISS index and documents")

FAISS index created with 8494 vectors
✅ Saved FAISS index and documents


## Getting Embeddings

In [ ]:
if "GEMINI_API_KEY" in st.secrets:
    api_key = st.secrets["GEMINI_API_KEY"]
else:
    api_key = os.getenv('GEMINI_API_KEY')

def load_rag_system():
    """Load FAISS index and documents"""
    
    index = faiss.read_index("data/sephora_faiss.index")
    
    # Load documents
    with open('data/sephora_documents.pkl', 'rb') as f:
        data = pickle.load(f)
    
    embedding_model = SentenceTransformer(
        data.get('embedding_model_name', 'all-MiniLM-L6-v2')
    )
    
    return index, data['documents'], embedding_model

In [ ]:
# Load system
index, documents, embedding_model = load_rag_system()
print(f"✅ Loaded {len(documents)} documents")

✅ Loaded 8494 documents


In [ ]:
def search_products(query, top_k=5):
    """Search for relevant products"""
    
    query_embedding = embedding_model.encode([query])
    
    # Search in FAISS
    distances, indices = index.search(
        query_embedding.astype('float32'), 
        top_k
    )
    
    # Get results with full document structure
    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        
        doc = documents[idx]  # This already has text + metadata
        results.append({
            'rank': i + 1,
            'text': doc['text'],
            'metadata': doc['metadata'],
            'distance': float(dist),
            'similarity_score': 1 / (1 + dist)  # Convert distance to similarity
        })
    
    return results
# Test it
results = search_products("Lip balm with SPF", top_k=3)

for r in results:
    print(f"\n{'='*60}")
    print(f"Rank {r['rank']} | Similarity: {r['similarity_score']:.3f}")
    print(f"Product Name: {r['metadata']['product_name']}")
    print(f"Brand Name:{r['metadata']['brand_name']}")
    print(f"Category: {r['metadata']['category']}")
    print(f"\nText Preview:")
    print(r['text'][:200] + "...")
  


Rank 1 | Similarity: 0.606
Product Name: Intense Therapy Lip Balm SPF 25
Brand Name:Jack Black
Category: Skincare

Text Preview:
Product Name: Intense Therapy Lip Balm SPF 25
Brand Name: Jack Black
Ingredients: Beeswax, Butyrospermum Parkii (Shea) Butter, C 18-38 Alkyl Hydroxystearol Stearate, Camellia Sinenesis (Green Tea) Lea...

Rank 2 | Similarity: 0.495
Product Name: Full Spectrum 360º Refreshing Water Cream Organic Face Sunscreen SPF 50
Brand Name:COOLA
Category: Skincare

Text Preview:
Product Name: Full Spectrum 360º Refreshing Water Cream Organic Face Sunscreen SPF 50
Brand Name: COOLA
Ingredients: None
On Sale: No
Reviews: 267.0 reviews
New: No
Limited Edition: No
Product Descrip...

Rank 3 | Similarity: 0.492
Product Name: Mini BB Blur Tinted Moisturizer Broad Spectrum SPF 30
Brand Name:tarte
Category: Makeup

Text Preview:
Product Name: Mini BB Blur Tinted Moisturizer Broad Spectrum SPF 30
Brand Name: tarte
Ingredients: Cyclopentasiloxane, Isododecane, Mica, Polysilicone-1

In [182]:
results = [r for r in results if r['similarity_score']>0.5]

In [183]:
results

[{'rank': 1,
  'text': 'Product Name: Intense Therapy Lip Balm SPF 25\nBrand Name: Jack Black\nIngredients: Beeswax, Butyrospermum Parkii (Shea) Butter, C 18-38 Alkyl Hydroxystearol Stearate, Camellia Sinenesis (Green Tea) Leaf Extract, Cyclohexasiloxane, Cyclopentasiloxane, Flavor, Lanolin, Microcrystalline Wax, Ozokerite, Persea Gratissima (Avocado) Oil, Polyglyceryl 3 Beeswax, Theobroma Cacao (Cocoa) Seed Butter, Tocopheryl Acetate.\nOn Sale: No\nReviews: 3595.0 reviews\nNew: No\nLimited Edition: No\nProduct Description: Without Parabens, Alcohol Free, SPF, Gluten Free\nVariations Available: Original - nan',
  'metadata': {'product_id': 'P12573',
   'brand_id': 3297,
   'rating': 4.7,
   'category': 'Skincare',
   'product_name': 'Intense Therapy Lip Balm SPF 25',
   'brand_name': 'Jack Black',
   'price_usd': 10.0},
  'distance': 0.6492201089859009,
  'similarity_score': 0.6063472028696619}]

## Connect To a LLM

In [ ]:


# # The client gets the API key from the environment variable `GEMINI_API_KEY`.
# client = genai.Client()

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents="Explain how AI works in a few words"
# )
# print(response.text)



AI learns patterns from data to make predictions or decisions.



In [ ]:
def generate_response(query, retrieved_docs, user_api_key):
    """Generate natural language response using LLM"""
  
    for attempt, key in enumerate([user_api_key, api_key]):
        if key is None:
            continue
    # Create context from retrieved documents
        try:
            context = "\n\n".join([
                f"Product {i+1}:\n{doc}" 
                for i, doc in enumerate(retrieved_docs)
            ])
            
            prompt = f"""You are a helpful Sephora beauty advisor. Based on the following products, 
        answer the customer's question naturally and recommend the best options.

        Customer Question: {query}

        Available Products:
        {context}

        Reply directly to the customer question.Provide a helpful recommendation with reasoning. Mention specific product names, prices, and key features."""
            client = genai.Client(api_key=key)
            response = client.models.generate_content(
            model="gemini-2.0-flash", contents=prompt
        )
            return response.text
        except Exception as e:
            if attempt == 0 :
                print(f"User Key Invalid {e}. Trying our key...")
                continue
            else:
                return f"Error: {str(e)}. Please provide a valid API key to continue"
    return "No valid API key available."

In [68]:
for i, doc in enumerate(results):
    print(doc['metadata'])

{'product_id': 'P453226', 'brand_id': 5987, 'category': 'Skincare', 'product_name': 'Bright-Eyed 100% Mineral Eye Cream SPF 40 PA+++', 'brand_name': 'Supergoop!'}
{'product_id': 'P395723', 'brand_id': 7091, 'category': 'Skincare', 'product_name': 'Classic Body Organic Sunscreen Spray SPF 50 Fragrance-Free', 'brand_name': 'COOLA'}
{'product_id': 'P468640', 'brand_id': 7091, 'category': 'Skincare', 'product_name': 'Full Spectrum 360º Refreshing Water Cream Organic Face Sunscreen SPF 50', 'brand_name': 'COOLA'}


In [31]:
query = "A lip balm with SPF Protection"
response = generate_response(query, results)

In [32]:
response

"Hi there! I can certainly help you find a lip balm with SPF protection.\n\nBased on what we have available, I highly recommend the **Jack Black Intense Therapy Lip Balm SPF 25** for $10. It has a fantastic rating of 4.68/5 with over 3500 reviews, which speaks to its popularity and effectiveness. This balm is formulated with moisturizing ingredients like shea butter, avocado oil, and cocoa seed butter to keep your lips soft and hydrated, while the SPF 25 provides essential sun protection. Plus, it's free of parabens and alcohol.\n"

## Build a RAG Chatnot

In [ ]:
def rag_chatbot(query, top_k=3, user_api_key = None):
    """Complete RAG pipeline"""
    
    print(f"🔍 Searching for: {query}")
    
    # 1: Retrieve relevant documents
    retrieved_docs = search_products(query, top_k=top_k)
    # Removing Documents too dissimilar 
    retrieved_docs= [r for r in retrieved_docs if r['similarity_score']>0.4]

    if len(retrieved_docs) > 0:
        print(f"✅ Found {len(retrieved_docs)} relevant products")
    else:
        print(f"❌ No relevant products")
        return {
            'query' : query,
            'response' : 'Sorry no products match this description in our catalogue',
        }
    
    # 2: Generate response
    response = generate_response(query, retrieved_docs, user_api_key)
    
    return {
        'query': query,
        'response': response,
        'retrieved_products': [
            {
                'name': doc['metadata']['product_name'],
                'brand': doc['metadata']['brand_name'],
                'price': doc['metadata']['price_usd'],
                'rating': doc['metadata']['rating']
            }
            for doc in retrieved_docs
        ]
    }
# Test it!
result = rag_chatbot("I need an affordable moisturizer for oily skin")
print("\n📝 Response:")
print(result['response'])
print("\n🛍️ Products considered:")
for p in result['retrieved_products']:
    print(f"- {p['name']} by {p['brand']} (${p['price']})| Rating: {p['rating']}/5")

🔍 Searching for: I need an affordable moisturizer for oily skin
✅ Found 3 relevant products

📝 Response:
Okay, I can help with that! Finding the right moisturizer for oily skin can be tricky, but there are definitely affordable options. Based on what we have, I'd recommend the **Peace Out Oil-Absorbing Pore Treatment Strips** for $19. While technically pore strips, it contains ingredients like DMAE Bitartrate and Vitamin A, perfect for oily skin and fighting pores!


🛍️ Products considered:
- Out of Trouble 10 Minute Mask to Rescue Problem Skin by Origins ($32.0)| Rating: 4.397/5
- Perfect Cleansing Oil by Shiseido ($35.0)| Rating: 4.4514/5
- Oil-Absorbing Pore Treatment Strips by Peace Out ($19.0)| Rating: 3.975/5
